In [1]:
# Load library
import pandas as pd
import numpy as np
from functools import reduce
from lib.util import fetch_tweets
from sklearn import model_selection, preprocessing, metrics

event = "germanwings-crash"

In [2]:
df = fetch_tweets(event)

In [3]:
df.head()

,is_rumor,thread,in_reply_tweet,event,text,tweet_id,is_source_tweet,in_reply_user,user_id,hashtags_count,...,mentions_count,user.tweets_count,user.verified,user.followers_count,user.friends_count,hasperiod,number_punct,negativewordcount,positivewordcount,sentimentscore
0,0,580319983676313601,NaN,germanwings-crash,#Últimahora Es tracta d'un aparell de Germanwi...,580319983676313601,1,NaN,8330472,1,...,0,107042,1,179430,74,1,11,1,0,0.0
1,0,580319983676313601,5.803199836763136e+17,germanwings-crash,“@324cat: #Últimahora Es tracta d'un aparell d...,580322851850461184,0,8330472.0,2307392966,1,...,1,2076,0,988,1782,1,16,1,0,0.0
2,0,580319983676313601,5.803228518504612e+17,germanwings-crash,@senyera @324cat 148 pasajeros 142+6,580323127089082368,0,2307392966.0,2535310842,0,...,2,701,0,62,121,0,11,0,0,0.0
3,0,580319983676313601,5.803231270890824e+17,germanwings-crash,@senyera @324cat dice el ministro frances que ...,580325737619685377,0,2535310842.0,2535310842,0,...,2,701,0,62,121,5,8,0,0,0.0
4,0,580321203757387776,NaN,germanwings-crash,Route map now being shared by http://t.co/LZkO...,580321203757387776,1,NaN,92771309,1,...,0,11447,1,20839,1354,3,19,0,0,0.0


In [4]:
Y = []
target = df['sentimentscore']
for val in target:
    if(val <= 0):
        Y.append('negative')
    else:
        Y.append('positive')

In [5]:
df['sentimen'] = Y

In [6]:
Y = []
target = df['is_rumor']
for val in target:
    if(val == 1):
        Y.append('rumor')
    else:
        Y.append('non-rumor')

In [7]:
df['rumor'] = Y

In [9]:
df_stack=df.groupby(['rumor','sentimen']).size().reset_index()
df_stack['Percentage']=df.groupby(['rumor',
       'sentimen']).size().groupby(level=0).apply(lambda 
        x:100 * x/float(x.sum())).values
df_stack.columns= ['rumor', 'sentimen', 'Counts', 
       'Percentage']
df_stack['Percentage'] = df_stack['Percentage'].map('{:,.2f}%'.format)

In [10]:
import plotly.express as px
fig2=px.bar(df_stack, x='rumor', y='Percentage',
     color='sentimen', barmode   ='stack',text=df_stack['Percentage'])
fig2.update_layout(title = "Sentiment Score' Distribution on Rumor and Non-rumor", 
      xaxis_title = event, yaxis_title =  
      'Percentage', width = 600, height = 400)
fig2.show()